# Setting up Enviornment

In [1]:
# Setting up api and enviornment
import os
import getpass
import asyncio
from dotenv import load_dotenv

# Setting Enviorment
ENV_PATH = "/Users/divyanshusinghania/Documents/Github/LangChain/.env"
load_dotenv(ENV_PATH)

if not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


# Document Creator

In [2]:
# Here we can define what an Document is in the code
# Used for in memory operations
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

# Data Loaders in LangChain

In [2]:
# Document Loaders - These are to use to connect and load the data
# - Types:
# - File (CSV, JSON, EXCEL) Based, Database, API Loader -> Sturctured Data Loader
# - File (TEXT, PDF PLUMBER, UNSTRUCTURED PDF), Web Pages -> Unstruvtured Text Loader
# - Cloud Based (S3, GCS, AZURE), Enterprise Knowledge Base -> Specialized and cloud
from langchain_community.document_loaders import PyPDFLoader

file_path = r"/Users/divyanshusinghania/Documents/Github/Reaserch Papers/2503.11651v1.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

20


In [3]:
print(f"{docs[0].page_content}\n")
print(docs[0].metadata)

VGGT: Visual Geometry Grounded Transformer
Jianyuan Wang1,2 Minghao Chen1,2 Nikita Karaev1,2 Andrea Vedaldi1,2
Christian Rupprecht1 David Novotny2
1Visual Geometry Group, University of Oxford 2Meta AI
…
Figure 1. VGGT is a large feed-forward transformer with minimal 3D-inductive biases trained on a trove of 3D-annotated data. It accepts
up to hundreds of images and predicts cameras, point maps, depth maps, and point tracks for all images at once in less than a second, which
often outperforms optimization-based alternatives without further processing.
Abstract
We present VGGT, a feed-forward neural network that di-
rectly infers all key 3D attributes of a scene, including cam-
era parameters, point maps, depth maps, and 3D point
tracks, from one, a few, or hundreds of its views. This
approach is a step forward in 3D computer vision, where
models have typically been constrained to and special-
ized for single tasks. It is also simple and efficient, re-
constructing images in under one se

# Chunking/Splitting

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

119

In [30]:
print(all_splits[0].metadata)
print(all_splits[0].page_content)

{'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-03-17T01:12:06+00:00', 'author': '', 'keywords': '', 'moddate': '2025-03-17T01:12:06+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/divyanshusinghania/Documents/Github/Reaserch Papers/2503.11651v1.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1', 'start_index': 0}
VGGT: Visual Geometry Grounded Transformer
Jianyuan Wang1,2 Minghao Chen1,2 Nikita Karaev1,2 Andrea Vedaldi1,2
Christian Rupprecht1 David Novotny2
1Visual Geometry Group, University of Oxford 2Meta AI
…
Figure 1. VGGT is a large feed-forward transformer with minimal 3D-inductive biases trained on a trove of 3D-annotated data. It accepts
up to hundreds of images and predicts cameras, point maps, depth maps, and point tracks for all images at once in less than a second, which
often outperforms optimizat

# Embedding Model

A big thing is that langsmith dont track embeddings by default, i will make changes afterwards for now its not tracking embeddings and vector store (Without Retriver)

## OpenAI Paid

In [5]:
# We are using OpenAI Embedding Model
# - "text-embedding-3-large" is .13 dollors per million tokens
# - "text-embedding-3-small" is .02 dollors per million tokens
from langchain_openai import OpenAIEmbeddings

embed_openai = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
Vector_OpenAI_1 = embed_openai.embed_query(all_splits[0].page_content)
Vector_OpenAI_2 = embed_openai.embed_query(all_splits[1].page_content)

assert len(Vector_OpenAI_1) == len(Vector_OpenAI_2)
print(f"Generated vectors of length {len(Vector_OpenAI_1)}\n")
print(Vector_OpenAI_1[:10])

Generated vectors of length 1536

[-0.027714015915989876, -0.009518314152956009, 0.032930199056863785, -0.026032162830233574, -0.03061460331082344, -0.05264931917190552, 0.02175440639257431, 0.028250260278582573, -0.05869423970580101, 0.01407638005912304]


## Ollama free (Local Serving, Laptop Intense)


### Setting Up Ollama Locally for LangChain

This note provides a quick revision guide to install and configure Ollama on your local machine so you can use the `OllamaEmbeddings` integration with LangChain.

---

### Prerequisites

- **Operating System:** macOS, Windows, or Linux (or WSL)
- **Python:** Version 3.11 or later
- **Ollama Installer:** Download from [Ollama's website](https://ollama.ai/)

---

### Installation Steps

#### 1. Download and Install Ollama

- **Download:** Visit [Ollama](https://ollama.ai/) and download the installer for your OS.
- **Install:** Follow the provided installation instructions.

#### 2. Start the Ollama Server

- **Open Terminal/Command Prompt**
- Run the command:
  ```bash
  ollama serve


In [7]:
from langchain_ollama import OllamaEmbeddings

embed_ollama = OllamaEmbeddings(
    model="llama3.2:latest"
)

In [8]:
Vector_Ollama_1 = embed_ollama.embed_query(all_splits[0].page_content)
Vector_Ollama_2 = embed_ollama.embed_query(all_splits[1].page_content)

assert len(Vector_Ollama_1) == len(Vector_Ollama_2)
print(f"Generated vectors of length {len(Vector_Ollama_1)}\n")
print(Vector_Ollama_1[:10])

Generated vectors of length 3072

[0.016136965, 0.0008328349, 0.016224116, 0.009177097, -0.02200352, -0.015884876, -0.03370971, 0.022160796, -0.0013032064, -0.007980467]


## Hugging Face Integration (Clashing with numpy will resolve later)

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings

embed_HF = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/Users/divyanshusinghania/Documents/Github/LangChain/Lang_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
Vector_HF_1 = embed_HF.embed_query(all_splits[0].page_content)
Vector_HF_2 = embed_HF.embed_query(all_splits[1].page_content)

assert len(Vector_HF_1) == len(Vector_HF_2)
print(f"Generated vectors of length {len(Vector_HF_1)}\n")
print(Vector_HF_1[:10])

Generated vectors of length 768

[-0.049434348940849304, 0.02994764968752861, 0.057593680918216705, 0.06480643898248672, -0.004581977613270283, 0.008656955324113369, -0.009640422649681568, -0.03884323313832283, -0.02391975186765194, -0.007224599830806255]


# Vector Store

A big thing is that langsmith dont track embeddings by default, i will make changes afterwards for now its not tracking embeddings and vector store (Without Retriver)

## In-Memory

In [11]:
# This is Langchain self implementation of Vector Store for In Memory
from langchain_core.vectorstores import InMemoryVectorStore

vector_store_OpenAI = InMemoryVectorStore(embed_openai)
vector_store_Ollama = InMemoryVectorStore(embed_ollama)
vector_store_HF = InMemoryVectorStore(embed_HF)

# Just to save some cost on OpenAI API
some_splits = all_splits[:2]
# len(all_splits)

ids_OpenAI = vector_store_OpenAI.add_documents(documents=some_splits)
ids_Ollama = vector_store_Ollama.add_documents(documents=some_splits)
ids_HF = vector_store_HF.add_documents(documents=some_splits)

In [12]:
some_splits

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-03-17T01:12:06+00:00', 'author': '', 'keywords': '', 'moddate': '2025-03-17T01:12:06+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/divyanshusinghania/Documents/Github/Reaserch Papers/2503.11651v1.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1', 'start_index': 0}, page_content='VGGT: Visual Geometry Grounded Transformer\nJianyuan Wang1,2 Minghao Chen1,2 Nikita Karaev1,2 Andrea Vedaldi1,2\nChristian Rupprecht1 David Novotny2\n1Visual Geometry Group, University of Oxford 2Meta AI\n…\nFigure 1. VGGT is a large feed-forward transformer with minimal 3D-inductive biases trained on a trove of 3D-annotated data. It accepts\nup to hundreds of images and predicts cameras, point maps, depth maps, and point tracks for all images at once in less than a s

## ChromaDB

In [ ]:
from langchain_chroma import Chroma

DB_path = r"/Users/divyanshusinghania/Documents/Github/LangChain/2.0) semantic_search_engine/chroma_langchain_db"

vector_store_OpenAI = Chroma(
    collection_name= "OpenAI",
    embedding_function= embed_openai,
    persist_directory= DB_path,  # Where to save data locally, remove if not necessary
)

vector_store_Ollama = Chroma(
    collection_name= "Ollama",
    embedding_function= embed_ollama,
    persist_directory= DB_path,  # Where to save data locally, remove if not necessary
)

vector_store_HF = Chroma(
    collection_name= "HuggingFace",
    embedding_function= embed_HF,
    persist_directory= DB_path,  # Where to save data locally, remove if not necessary
)

ids_OpenAI = vector_store_OpenAI.add_documents(documents=some_splits)
ids_Ollama = vector_store_Ollama.add_documents(documents=some_splits)
ids_HF = vector_store_HF.add_documents(documents=some_splits)

### Querying

In [17]:
# These are Syncronus

results_HF = vector_store_HF.similarity_search(
    "I have given you a Document, i wanted to know what is its name."
)

results_Ollama = vector_store_Ollama.similarity_search(
    "I have given you a Document, i wanted to know what is its name."
)

results_OpenAI = vector_store_OpenAI.similarity_search(
    "I have given you a Document, i wanted to know what is its name."
)

# This is Asyncronus

results = await vector_store_OpenAI.asimilarity_search("I have given you a Document, i wanted to know what is its name.")

print(results[0])

page_content='VGGT: Visual Geometry Grounded Transformer
Jianyuan Wang1,2 Minghao Chen1,2 Nikita Karaev1,2 Andrea Vedaldi1,2
Christian Rupprecht1 David Novotny2
1Visual Geometry Group, University of Oxford 2Meta AI
…
Figure 1. VGGT is a large feed-forward transformer with minimal 3D-inductive biases trained on a trove of 3D-annotated data. It accepts
up to hundreds of images and predicts cameras, point maps, depth maps, and point tracks for all images at once in less than a second, which
often outperforms optimization-based alternatives without further processing.
Abstract
We present VGGT, a feed-forward neural network that di-
rectly infers all key 3D attributes of a scene, including cam-
era parameters, point maps, depth maps, and 3D point
tracks, from one, a few, or hundreds of its views. This
approach is a step forward in 3D computer vision, where
models have typically been constrained to and special-
ized for single tasks. It is also simple and efficient, re-' metadata={'author': 

In [18]:
results_OpenAI = vector_store_OpenAI.similarity_search_with_score(
    "I have given you a Document, i wanted to know what is its name."
)
_, score = results_OpenAI[0]
print(results_OpenAI[0]) # -> 1536 size, paid
print(score)

results_HF = vector_store_HF.similarity_search_with_score(
    "I have given you a Document, i wanted to know what is its name."
)
_, score = results_HF[0]
print(results_HF[0]) # -> 768 size, free
print(score)

results_Ollama = vector_store_Ollama.similarity_search_with_score(
    "I have given you a Document, i wanted to know what is its name."
)
_, score = results_Ollama[0]
print(results_Ollama[0]) # -> 3072 size, free
print(score)

(Document(id='1309f11c-a775-4f2d-8b7b-2f380ca0a5e6', metadata={'author': '', 'creationdate': '2025-03-17T01:12:06+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-03-17T01:12:06+00:00', 'page': 0, 'page_label': '1', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': '/Users/divyanshusinghania/Documents/Github/Reaserch Papers/2503.11651v1.pdf', 'start_index': 0, 'subject': '', 'title': '', 'total_pages': 20, 'trapped': '/False'}, page_content='VGGT: Visual Geometry Grounded Transformer\nJianyuan Wang1,2 Minghao Chen1,2 Nikita Karaev1,2 Andrea Vedaldi1,2\nChristian Rupprecht1 David Novotny2\n1Visual Geometry Group, University of Oxford 2Meta AI\n…\nFigure 1. VGGT is a large feed-forward transformer with minimal 3D-inductive biases trained on a trove of 3D-annotated data. It accepts\nup to hundreds of images and predicts cameras, point maps, depth maps, and point tra

In [ ]:
# Used to confirm the tracking is going on 
# but ofcourse the embedding and vectors are not tracked as the are not runnables

from langsmith import utils
utils.tracing_is_enabled()

True

# Retrievers

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain

# Only will use HF for Explaination (not building an application here)
@chain
def retrieve_hf(query: str) -> List[Document]:
    return vector_store_HF.similarity_search(query, k=1)

retrieve_hf.batch(
    [
        "query 1",
        "query 2",
    ],
)
# -> getting an HuggingFace error saying process forked
# -> I think batch gives parllelism to retriver and hence the error



# retriever = vector_store_HF.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 1},
# )

# retriever.batch(
#     [
#         "query 1",
#         "query 2",
#     ],
# )


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[[Document(id='7b6019db-c7fc-4735-995b-7ae270b1c8a0', metadata={'author': '', 'creationdate': '2025-03-17T01:12:06+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-03-17T01:12:06+00:00', 'page': 0, 'page_label': '1', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': '/Users/divyanshusinghania/Documents/Github/Reaserch Papers/2503.11651v1.pdf', 'start_index': 794, 'subject': '', 'title': '', 'total_pages': 20, 'trapped': '/False'}, page_content='approach is a step forward in 3D computer vision, where\nmodels have typically been constrained to and special-\nized for single tasks. It is also simple and efficient, re-\nconstructing images in under one second, and still out-\nperforming alternatives that require post-processing with\nvisual geometry optimization techniques. The network\nachieves state-of-the-art results in multiple 3D tasks, in-\ncluding camera parame